<a href="https://colab.research.google.com/github/Bziukiewicz/tests/blob/master/binary_heart_failure_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install -q tensorflow

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Input, GRU, Lambda

In [5]:
normal= pd.read_csv('/content/ptbdb_normal.csv')
abnormal=pd.read_csv('/content/ptbdb_abnormal.csv')
normal.columns= list(range(len(normal.columns)))
abnormal.columns= list(range(len(abnormal.columns)))
print(abnormal.shape)
print(normal.shape)

(10505, 188)
(4045, 188)


In [6]:
all_data=pd.concat([normal, abnormal])


In [7]:
all_data.isnull().sum()

0      0
1      0
2      0
3      0
4      0
      ..
183    0
184    0
185    0
186    0
187    0
Length: 188, dtype: int64

In [8]:
data=all_data.copy()
target = pd.DataFrame(data.pop(187))

In [9]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [10]:
y_train.value_counts()
X_train.shape

(10912, 187)

In [11]:
y_test.value_counts()

187
1.0    2633
0.0    1005
Name: count, dtype: int64

In [12]:

from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)


In [13]:
model = Sequential()
model.add(Input(shape=[len(X_train.keys())]))
model.add(Lambda(lambda x: tf.expand_dims(x, axis=-1)))
model.add(GRU(128, return_sequences=True))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
            loss='binary_crossentropy',
              metrics=['accuracy'])

In [14]:
history = model.fit(X_train, y_train, epochs=30, validation_split=0.2, batch_size=32, callbacks=[es])

Epoch 1/30
273/273 ━━━━━━━━━━━━━━━━━━━━ 55s 193ms/step - accuracy: 0.7500 - loss: 0.5012 - val_accuracy: 0.7820 - val_loss: 0.4210
Epoch 2/30
273/273 ━━━━━━━━━━━━━━━━━━━━ 54s 199ms/step - accuracy: 0.7870 - loss: 0.4375 - val_accuracy: 0.8259 - val_loss: 0.3856
Epoch 3/30
273/273 ━━━━━━━━━━━━━━━━━━━━ 84s 208ms/step - accuracy: 0.8121 - loss: 0.3903 - val_accuracy: 0.8305 - val_loss: 0.3687
Epoch 4/30
273/273 ━━━━━━━━━━━━━━━━━━━━ 79s 198ms/step - accuracy: 0.8388 - loss: 0.3559 - val_accuracy: 0.8479 - val_loss: 0.3417
Epoch 5/30
273/273 ━━━━━━━━━━━━━━━━━━━━ 83s 202ms/step - accuracy: 0.8582 - loss: 0.3198 - val_accuracy: 0.8388 - val_loss: 0.3538
Epoch 6/30
273/273 ━━━━━━━━━━━━━━━━━━━━ 79s 191ms/step - accuracy: 0.8787 - loss: 0.2913 - val_accuracy: 0.8804 - val_loss: 0.2687
Epoch 7/30
273/273 ━━━━━━━━━━━━━━━━━━━━ 83s 194ms/step - accuracy: 0.8965 - loss: 0.2396 - val_accuracy: 0.8846 - val_loss: 0.2508
Epoch 8/30
273/273 ━━━━━━━━━━━━━━━━━━━━ 85s 204ms/step - accuracy: 0.8952 - loss: 0

In [16]:
results=model.evaluate(X_test, y_test, verbose=0)
print(f'Accuracy: {results[1]:.4f}')
print(f'Loss: {results[0]:.4f}')

Accuracy: 0.9733
Loss: 0.0972
